# KPop song clustering

### Objective: Used unsupervised learning to cluster korean pop songs that I like (in playlist). Compare with Anime playlist. Compare with "iconic" KPop playlist

### Methodology: 1) Create dataset with track name, artist, audio features, playlist name, like y/n

In [44]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import time
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [24]:
#Authorize credentials
#Note that with the free version, API calls are limited
cid = "c433ddf327d64bbcbae13b46dc5a56c9" 
secret = "ed9ffb3511ff4bb7910d520564ddbd79" 
user_id = '22pjtm23jl5h4qq4srfwxwxqa' #my playlists
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [25]:
def getplayListDict(user_id):
    playlists = sp.user_playlists(user_id)
    names = [x['name'] for x in playlists['items']]
    ids = [x['id'] for x in playlists['items']]
    return dict(zip(names, ids))

In [26]:
def getTrackInfoFromPlaylist(user_id, playlist_id):
    offset = 0
    songs = []
    genres = []
    while True:
        content = sp.user_playlist_tracks(user_id, playlist_id, limit = 50, offset=offset)
        songs += content['items']
        if content['next'] is not None:
            offset+=50
        else:
            break
            
    names = [x['track']['name'] for x in songs]
    ids = [x['track']['id'] for x in songs]
    artists = [x['track']['artists'][0]['name'] for x in songs]
    popularity = [x['track']['popularity'] for x in songs]
    
    df = pd.DataFrame({'Track Name': names, 'Track ID': ids, 'Artist': artists, 'Track popularity': popularity})
    return df

In [27]:
def getArtistInfoFromPlaylist(user_id, playlist_id):
    offset = 0
    songs = []
    genres = []
    while True:
        content = sp.user_playlist_tracks(user_id, playlist_id, limit = 50, offset=offset)
        songs += content['items']
        if content['next'] is not None:
            offset+=50
        else:
            break
            
    artist_ids = [x['track']['artists'][0]['id'] for x in songs]
    artist_ids = filter(None, artist_ids)
    
    n_calls = 0
    artists = []
    for artist_id in artist_ids:
        try:
            content = sp.artist(artist_id)
            artists.append(content)
        except Exception as e:
            print(artist_id)
            print(e)
    try:
        followers = [x['followers']['total'] for x in artists]
        genres = [x['genres'] for x in artists]
        popularity = [x['popularity'] for x in artists] 
        name = [x['name'] for x in artists]
        ids = [x['id'] for x in artists]
    except Exception as e:
        print(e)
    
    df = pd.DataFrame({'Artist ID':ids, 'Artist': name,'Artist Followers':followers, 'Artist genres': genres, 'Artist popularity':popularity})
    return df

In [28]:
def getAudioFeatures(track_id_list, limit=50):
    features = []
    index = 0
    while index < len(track_id_list):
        features += sp.audio_features(track_id_list[index:index+limit])
        index += limit
    return features

In [29]:
my_playlist = getplayListDict(user_id)

In [30]:
my_playlist

{'Kpop 2015-2021': '5y7T1K8bicDH8rpU1uXDCK',
 'BEST KPOP 2021 - TOP HITS': '2EoheVFjqIxgJMb8VnDRtZ',
 'Kpop girl group songs that were a cultural reset': '3UeJumqoZFSXqRQ9b3F6SB',
 'KPOP': '75rPT4TszZleyYpYqzAZgf',
 'anime covers': '3duEwjd8M5Hugz8UkAFQdC',
 'lit anime music': '3LvTqHT3lOBomXlfqmUm8M',
 'Shounen Anime Vibes (Anime Opening & Ending)': '5MjsmYfqrFrg0AMGKhg1JI'}

In [31]:
concat_df_tracks = []
concat_df_artists = []
for playlist in my_playlist.keys():
    playlist_df_tracks = getTrackInfoFromPlaylist(user_id, my_playlist[playlist])
    playlist_df_artists = getArtistInfoFromPlaylist(user_id, my_playlist[playlist])
    
    playlist_df_tracks['Playlist'] = playlist
    playlist_df_artists['Playlist'] = playlist
    
    concat_df_tracks.append(playlist_df_tracks)
    concat_df_artists.append(playlist_df_artists)
    
tracks_df = pd.concat(concat_df_tracks)
artists_df = pd.concat(concat_df_artists)

In [32]:
#remove none
tracks_df.dropna(inplace=True)
print(tracks_df.shape)
tracks_df.head()

(1683, 5)


,Track Name,Track ID,Artist,Track popularity,Playlist
0,Hobgoblin,2JTjDYA0vfx8sp5XMTbeqU,CLC,0.0,Kpop 2015-2021
1,RHYTHM TA,374WyJeaSR9Cmq4kuFzoqu,iKON,0.0,Kpop 2015-2021
2,Liar,67D8AmrRyahBZNKzPBBsJG,CLC,0.0,Kpop 2015-2021
3,Oh NaNa (Hidden. HUR YOUNG JI),1ZgfAxHQCXLt8o1VXEHHAt,KARD,53.0,Kpop 2015-2021
4,Don't Recall,4b1sQpvL7QVgamRZ74F1oA,KARD,53.0,Kpop 2015-2021


In [33]:
artists_df.dropna(inplace=True)
artists_df.drop_duplicates(inplace=True, subset='Artist')
print(artists_df.shape)
artists_df.head()

(524, 6)


,Artist ID,Artist,Artist Followers,Artist genres,Artist popularity,Playlist
0,6QyO41KctzGc70mVaVnXQO,CLC,978300.0,"[k-pop, k-pop girl group, korean city pop]",59.0,Kpop 2015-2021
1,5qRSs6mvI17zrkJpOHkCoM,iKON,3125872.0,"[k-pop, k-pop boy group]",65.0,Kpop 2015-2021
3,2JhAlkmukNvarUpGhTFXUQ,KARD,1470636.0,[k-pop],58.0,Kpop 2015-2021
7,7LOmc7gyMVMOWF8qwEdn2X,SF9,1324598.0,"[k-pop, k-pop boy group]",59.0,Kpop 2015-2021
11,7f4ignuCJhLXfZ9giKT7rH,NCT 127,4534752.0,"[k-pop, k-pop boy group]",75.0,Kpop 2015-2021


In [34]:
audio_features = pd.DataFrame(getAudioFeatures(tracks_df['Track ID'].tolist()))

In [35]:
audio_feature_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

In [36]:
df_merged_features = tracks_df.join(audio_features, how='inner', lsuffix='Track ID', rsuffix = 'id')
df_merged_features.shape

(1683, 23)

In [37]:
df_merged_features = df_merged_features.merge(artists_df, how='inner', left_on='Artist', right_on = 'Artist')
df_merged_features.shape

(1683, 28)

In [38]:
df_merged_features.head()

,Track Name,Track ID,Artist,Track popularity,Playlist_x,danceability,energy,key,loudness,mode,...,uri,track_href,analysis_url,duration_ms,time_signature,Artist ID,Artist Followers,Artist genres,Artist popularity,Playlist_y
0,Hobgoblin,2JTjDYA0vfx8sp5XMTbeqU,CLC,0.0,Kpop 2015-2021,0.621,0.873,11,-2.888,0,...,spotify:track:2JTjDYA0vfx8sp5XMTbeqU,https://api.spotify.com/v1/tracks/2JTjDYA0vfx8...,https://api.spotify.com/v1/audio-analysis/2JTj...,210478,4,6QyO41KctzGc70mVaVnXQO,978300.0,"[k-pop, k-pop girl group, korean city pop]",59.0,Kpop 2015-2021
1,HELICOPTER,2hJS7BdEaH9Jo8MeJujsHT,CLC,66.0,KPOP,0.621,0.873,11,-2.888,0,...,spotify:track:2JTjDYA0vfx8sp5XMTbeqU,https://api.spotify.com/v1/tracks/2JTjDYA0vfx8...,https://api.spotify.com/v1/audio-analysis/2JTj...,210478,4,6QyO41KctzGc70mVaVnXQO,978300.0,"[k-pop, k-pop girl group, korean city pop]",59.0,Kpop 2015-2021
2,Liar,67D8AmrRyahBZNKzPBBsJG,CLC,0.0,Kpop 2015-2021,0.735,0.829,6,-3.510,0,...,spotify:track:67D8AmrRyahBZNKzPBBsJG,https://api.spotify.com/v1/tracks/67D8AmrRyahB...,https://api.spotify.com/v1/audio-analysis/67D8...,197532,4,6QyO41KctzGc70mVaVnXQO,978300.0,"[k-pop, k-pop girl group, korean city pop]",59.0,Kpop 2015-2021
3,BLACK DRESS,4g73tmQQc6pSfe4NMXKrZD,CLC,56.0,KPOP,0.735,0.870,1,-1.955,1,...,spotify:track:7DWkKAq8wWZk6WMu6tdqe2,https://api.spotify.com/v1/tracks/7DWkKAq8wWZk...,https://api.spotify.com/v1/audio-analysis/7DWk...,187382,4,6QyO41KctzGc70mVaVnXQO,978300.0,"[k-pop, k-pop girl group, korean city pop]",59.0,Kpop 2015-2021
4,Devil,3aG0gbuvZ1VeVm68d4J6Yq,CLC,58.0,KPOP,0.715,0.760,2,-3.353,0,...,spotify:track:3C0bt8wqscqmNTtwUuOzDZ,https://api.spotify.com/v1/tracks/3C0bt8wqscqm...,https://api.spotify.com/v1/audio-analysis/3C0b...,206351,4,6QyO41KctzGc70mVaVnXQO,978300.0,"[k-pop, k-pop girl group, korean city pop]",59.0,Kpop 2015-2021


In [39]:
df_merged_features.drop(columns=['Playlist_y'], inplace=True)

In [40]:
df_merged_features = df_merged_features.rename(columns={'Playlist_x': 'Playlist'})

In [41]:
#Renaming playlists
df_merged_features['Playlist'].replace('Kpop girl group songs that were a cultural reset', 'Kpop-iconic', inplace=True)
df_merged_features['Playlist'].replace('KPOP', 'Kpop-myselection', inplace=True)
df_merged_features['Playlist'].replace('BEST KPOP 2021 - TOP HITS', 'Kpop-2021', inplace=True)
df_merged_features['Playlist'].replace('lit anime music', 'Anime-myselection', inplace=True)
df_merged_features['Playlist'].replace(r'Shounen Anime Vibes (Anime Opening & Ending)', 'Anime-iconic', inplace=True)

df_merged_features['Preference'] = np.where(df_merged_features['Playlist'].str.contains('my'), 'Mine', 'Other')
df_merged_features['Type'] = np.where(df_merged_features['Playlist'].str.contains('Kpop'), 'Kpop', 'Anime')

In [45]:
mlb = MultiLabelBinarizer()

In [54]:
genres = pd.DataFrame(mlb.fit_transform(df_merged_features['Artist genres']), columns = mlb.classes_, index=df_merged_features.index)

In [58]:
df_merged_features = df_merged_features.merge(genres, left_index=True, right_index=True)

In [59]:
df_merged_features.to_csv('kpop_data_science/playlist_features.csv', index=None)

In [61]:
sp.user_playlists(user_id)

{'href': 'https://api.spotify.com/v1/users/22pjtm23jl5h4qq4srfwxwxqa/playlists?offset=0&limit=50',
 'items': [{'collaborative': False,
   'description': 'My favorites of Kpop from 2015 to 2021.',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/5y7T1K8bicDH8rpU1uXDCK'},
   'href': 'https://api.spotify.com/v1/playlists/5y7T1K8bicDH8rpU1uXDCK',
   'id': '5y7T1K8bicDH8rpU1uXDCK',
   'images': [{'height': None,
     'url': 'https://i.scdn.co/image/ab67706c0000bebb83e8eb42ce16323ee7ee08a4',
     'width': None}],
   'name': 'Kpop 2015-2021',
   'owner': {'display_name': 'yamihallyu',
    'external_urls': {'spotify': 'https://open.spotify.com/user/yamihallyu'},
    'href': 'https://api.spotify.com/v1/users/yamihallyu',
    'id': 'yamihallyu',
    'type': 'user',
    'uri': 'spotify:user:yamihallyu'},
   'primary_color': None,
   'public': True,
   'snapshot_id': 'MTA0MSxkNGM5ODE2ZmQ5NTBjNjUzMjEyNmZlYmY3NzNiZmU2ZGMzZjlmMDFj',
   'tracks': {'href': 'https://api.spotify.com/v1/